In [8]:
# from reso_extract import ms_graph
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

import sys
from pathlib import Path

# Define the path you want to add
new_path = Path("C:\\Users\\andmn\\PycharmProjects\\kverse")

# Convert Path object to string and append it to sys.path
sys.path.append(str(new_path))
from selenium.webdriver.support.wait import WebDriverWait
from kdags.assets.reparation.reso.web_driver import initialize_driver, login_to_reso
from kdags.assets.reparation.reso.document_extractor import (
    extract_docs_data,
    click_see_service_order,
)
from kdags.assets.reparation.reso.file_processor import process_docs_data
from kdags.assets.reparation.reso.web_utils import wait_for_page_load, ensure_clickable
from kdags.assets.reparation.reso.file_utils import save_file_locally
from kdags.assets.reparation.reso.config import DEFAULT_WAIT
import pandas as pd
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC

driver = initialize_driver()
wait = WebDriverWait(driver, DEFAULT_WAIT)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
login_to_reso(driver, wait)

True

In [10]:
service_order_df = pd.read_excel(
    r"C:\Users\andmn\PycharmProjects\kbooks\banco_parrillas_blower.xlsx"
)
service_order_df

,service_order,customer_work_order,component_serial,changeout_date,subcomponent_name
0,14134107,18118435,EE07020083A,2023-01-11,blower
1,14134929,18118983,EE99100156,2023-02-16,blower
2,14135461,18119426,EE12100031,2023-03-21,blower
3,14135762,18119693,EE04070186,2023-03-30,blower
4,14136123,18119819,EE13111405,2023-04-16,blower
...,...,...,...,...,...
68,14151150,18130120,EE14091200,2024-11-22,blower
69,14151046,18130428,EE06050308,2024-12-08,blower
70,14151789,18130771,EE13111350,2024-12-27,blower
71,14151972,18130827,EE21060597,2025-01-09,blower


In [ ]:
# Ensure the 'processed' column exists (default False)
if "processed" not in service_order_df.columns:
    service_order_df["processed"] = False


# Iterate over service orders that are not yet processed
for idx, ir in service_order_df.iterrows():
    if ir.processed:
        continue  # Skip already processed rows

    print(f"\nProcessing Row {idx} for Service Order: {ir.service_order}")

    # Wait for page to be fully loaded
    wait_for_page_load(wait)

    # Wait for and enter the service order code
    input_field = wait.until(EC.presence_of_element_located((By.ID, "codigoBuscar")))
    input_field.clear()
    input_field.send_keys(ir.service_order)

    # Click search button
    search_button = wait.until(EC.element_to_be_clickable((By.ID, "buttonBuscar")))
    ensure_clickable(driver, search_button)
    search_button.click()

    # Wait for results to load
    wait_for_page_load(wait)
    wait.until(EC.presence_of_element_located((By.ID, "WebGrid")))

    # Click the 'See service order' button
    click_see_service_order(wait)

    # Wait for page load after clicking service order
    wait_for_page_load(wait)

    # Click the Documents tab and wait for it to load
    docs_tab = wait.until(EC.element_to_be_clickable((By.ID, "v-pills-documentos-tab")))
    ensure_clickable(driver, docs_tab)
    docs_tab.click()
    wait_for_page_load(wait)

    # Extract the docs data from the documents section
    docs_data = extract_docs_data(driver, wait)

    # Process the extracted docs data
    processed_df = process_docs_data(
        docs_data, ir.subcomponent_name, ir.component_serial
    ).query("file_type=='INFORMES'")

    # Iterate over each document row and upload
    for jdx, row in processed_df.iterrows():
        result_url = save_file_locally(
            target_folder=row["target_folder"],
            file_name=row["file_name"],
            blob_url=row["blob_url"],
        )
        print(f"Document {jdx}: Uploaded successfully: {result_url}")

    # Click the close button
    close_button = wait.until(EC.element_to_be_clickable((By.ID, "closePCI")))
    ensure_clickable(driver, close_button)
    close_button.click()

    # Wait for page to return to initial state
    wait_for_page_load(wait)

    # Mark the service order as processed
    service_order_df.loc[idx, "processed"] = True


Processing Row 0 for Service Order: 14134107
